# Guide01

## Introduction

This guide explains how to obtain SpatialIDs within a certain distance of a given line. We'll review the following functions:
- GetSpatialIdsWithinRadiusOfLine()
- GetNspatialIdsAroundVoxcels()
- FitClearanceAroundExtendedSpatialID()

### External Data and License Notice
This guide will use geodetic coordinates from [OpenStreetMap](openstreetmap.org/copyright); please note that the data is available under the Open Database License. Please refer to the link for more information. The following code is not part of OpenStreeMap; it's provided by Trajectory under the MIT license. Please feel free to learn and contribute!

### About Jupyter Notebooks with Go

This notebook uses [GoNB](https://github.com/janpfeifer/gonb) for the Go integration. Please see the repo for instruction on how to set it up so you can run this notebook yourself.

As explained [here](https://github.com/janpfeifer/gonb/blob/main/examples/tutorial.ipynb), the `%%` shortcut is a feature of GoNB. It wraps subsequent code in a main() function that allows for things like a `fmt.Print()` call. If you're copying the code into a standard `.go` file, please exclude `%%`.

In [17]:
// setup cell: import packages
import (
	"fmt"
	"math"
	"github.com/go-gl/mathgl/mgl64"
    "github.com/trajectoryjp/geodesy_go/coordinates"
	closest "github.com/trajectoryjp/closest_go"
	geodesy "github.com/trajectoryjp/geodesy_go/coordinates"
	"github.com/trajectoryjp/spatial_id_go/v2/common/errors"
	"github.com/trajectoryjp/spatial_id_go/v2/common/object"
	"github.com/trajectoryjp/spatial_id_go/v2/shape"
	
)

%%
fmt.Print("Setup complete")


Setup complete

## Part 1: Simple Use Case

It's sakura season here in Japan -- which means there's no better time to learn about the inner workings of SpatialIDs! Let's go to a popular [hanami](https://en.wikipedia.org/wiki/Hanami) spot in Tokyo: [Yoyogi Park](https://en.wikipedia.org/wiki/Yoyogi_Park).

Bringing a baseball is never a bad idea. But it's crowded. Let's find a straight-line area where there aren't any people.

First we use `object.NewPoint()` to declare our two points.


In [25]:
var (
	pointA *object.Point
	pointB *object.Point
	error error
) 


%%
pointA, error := object.NewPoint(139.69441, 35.67269, 0)
if error != nil {
	fmt.Print(error)
	}
pointB, error := object.NewPoint(139.69516, 35.67157, 0)
if error != nil {
	fmt.Print(error)
}

fmt.Printf("pointA: %v\npointB: %v", *pointA, *pointB)

pointA: {139.69441 35.67269 0}
pointB: {139.69516 35.67157 0}